<a href="https://colab.research.google.com/github/adarsadhikari/forest-fire-prediction_CNN/blob/main/Forest_fire_prediction_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
img_size=128
def load_process_image(imgpath):
  img=cv2.imread(imgpath)
  img=cv2.resize(img,(img_size,img_size))
  img=img/255.0
  return img
imgfolder='/content/image'
wildfire = []
for img_name in os.listdir(imgfolder):
  imgpath = os.path.join(imgfolder, img_name)
  processed_img = load_process_image(imgpath)
  wildfire.append(processed_img)
#wildfire=[load_process_image(os.path.join(imgfolder,img)) for img in os.listdir(imgfolder)]
wildfire=np.array(wildfire)
print(wildfire.shape)

(7, 128, 128, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen=ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
augmented_images=[]
for img in wildfire:
  img=img.reshape((1,)+img.shape)
  for batch in datagen.flow(img,batch_size=1):
    augmented_images.append(batch[0])
    if len(augmented_images)>=14:
      break
augmented_images=np.array(augmented_images)
all_img=np.concatenate((wildfire,augmented_images), axis=0)
print(f"Total images after augmentation: {len(all_img)}")


Total images after augmentation: 27


In [ ]:
labels = np.ones(len(all_img))
print(labels.shape)

(27,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
model=Sequential([
    Conv2D(32,(3,3), activation='relu', input_shape=(img_size,img_size,3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64,(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128,(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(all_img,labels,test_size=0.2,random_state=1)
history=model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),batch_size=4)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 302ms/step - accuracy: 0.7145 - loss: 0.2723 - val_accuracy: 1.0000 - val_loss: 6.9931e-18
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 1.0000 - loss: 9.8307e-22 - val_accuracy: 1.0000 - val_loss: 3.4552e-38
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 1.00

In [ ]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Loss: 0.0
Accuracy: 1.0


In [ ]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(128,128))  # Resize the image
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)  # Expand dimensions for batch size
    img = img / 255.0  # Normalize

    prediction = model.predict(img)

    if prediction[0][0] > 0.5:
        print(f"Prediction: Fire Detected with confidence {prediction[0][0]:.2f}")
    else:
        print(f"Prediction: No Fire Detected with confidence {1 - prediction[0][0]:.2f}")

In [ ]:
path='/content/test/test1.jpg'

In [ ]:
predict_image(path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Prediction: Fire Detected with confidence 1.00
